<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/POI-case-study-part1-v2-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Case Study: Accessibility - Part 1

The City of Columbus has put together a collection of points of interest for different functional categories. This data can be viewed at their interactive map https://opendata.columbus.gov/datasets/columbus::points-of-interest/explore. The description of the data can be found at https://maps2.columbus.gov/arcgis/rest/services/Schemas/PointsOfInterest/MapServer/10. The raw geojson file has been posted at https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson, which can also be downloaded from their interactive map page. We will use this geojson file in this tutorial.

The goal of this tutorial is to demonstrate the use of Python for spatial data analysis. More specifically, we will examine how residents in Franklin county can access some of the services represented in this data set. Of course the data is probably incomplete because it may not fully cover areas outside the city of Columbus and some points of interest may not be included in the data either. The main purpose is to demonstrate the use of data and Python coding to analyze accessibility. 

It is best to treat this tutorial as a working notebook (it is indeed a Jupyter Notebook). Only the code for the beginning is included. Students will need to type the code by following the video or in-class instruction. Markdown cells should also be used to explain the code.

After finishing this tutorial, students should answer the following questions (with code and results from the code):

1. How many types of points of interest are represented in the data and how many instances are there in each type?
2. How the POIs are distributed on a map? 

## Environment and Data

Since we are going to run every piece of code on a Jupyter Notebook or Jupyter Lab server, we get direct access to the cloud. Luckily everything we use, the data and library will all be available in the cloud. 

First, let's make sure we have the geometry module used so far in this class cloned in our current working environment. After the following two lines of code, a colder called geom will be created and every file in the geom repo will be cloned to that folder. This allows us to directly use the geom modules without any additional process.

In [ ]:
!rm -rf geom
!git clone https://github.com/gisalgs/geom.git

The following two libraries are essential to access online data files and handle JSON and GeoJSON files. Here we will use a more popular (and more stable) module called `requests` (in 2025) to access online files. 

In [ ]:
# import urllib.request as request
import requests
import json 

### Population data

In addition to the POIs, we will also need to have the population data at the census block group level for Franklin county. 

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'

blkgrps = json.loads(requests.get(url).text)

len(blkgrps['features'])

We can also try the `urllib` module as follows, but it may run into some issues on macOS. 

```python
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'
with request.urlopen(url) as response:
    blkgrps = json.loads(response.read())

len(blkgrps['features'])
```

In [ ]:
blkgrps['features'][0]['properties'].keys()

We are probably very eager to check how many of these census block group polygons are multiplygons. Actually all of them are!

In [ ]:

sum([f['geometry']['type'] == 'MultiPolygon' for f in blkgrps['features']])

So how many are actually having multiple parts? There are only two.

In [ ]:
sum([len(f['geometry']['coordinates'])>1 for f in blkgrps['features']])

Finally, are there any polygons (or multipolygons) with holes? There is actually one. So things will get complicated when we do drawing, etc.

In [ ]:
sum([len(f['geometry']['coordinates'][0])>1 for f in blkgrps['features']])

Let's print out some info about this one.

In [ ]:
for f in blkgrps['features']:
    if len(f['geometry']['coordinates'][0])>1:
        print(f['properties']['TRACT'], f['properties']['BLKGRP'])

We will use the population data next time.

### POI data

Now we get the POI data, which can be either from the github data repo as below. Or we can download the data from the city's link and save it to an online location that is publicly accessible and use the link from there. 

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson'

poi = json.loads(requests.get(url).text)

len(poi['features'])

Explore the types of geometry in the POI data.

In [ ]:
geom_types = []
for p in poi['features']:
    t = p['geometry']['type']
    if t not in geom_types:
        geom_types.append(t)

geom_types

Now we know all the features are of the same type: Point. Let's know explore the attributes a little bit before we draw the map.

In [ ]:
p['properties']

#### Getting the types



In [ ]:
poi_types = {}
for p in poi['features']:
    t = p['properties']['POI_TYPE']
    if t not in poi_types:
        poi_types[t] = 1
    else:
        poi_types[t] += 1

print('Total number of types:', len(poi_types))
for t in poi_types:
    print(f'{t:>10}: {poi_types[t]}')

Make sure to follow the instructions to complete the remaining of this notebook.